trying to get NFL odds from yahoo... of course it's totally different than NBA, because yahoo

these urls contain the line, but not the betting percentages.
https://graphite.sports.yahoo.com/v1/query/shangrila/moduleGame?lang=en-US&region=US&ysp_platform=next-app-sports&gameId=nfl.g.20251225028

for comparison, the NBA one with all the gold is:

    return f"https://sports.yahoo.com/site/api/resource/sports.graphite.gameOdds;dataType=graphite;endpoint=graphite;gameIds={game_id}"

I wonder if this works


https://sports.yahoo.com/site/api/resource/sports.graphite.gameOdds;dataType=graphite;endpoint=graphite;gameIds=nfl.g.20251225028


wait, holy cow, it does!

here is a sample NFL game ID: nfl.g.20251225028

In [1]:
import scrape_yahoo

In [2]:
SAMPLE_NFL_GAME = "nfl.g.20251225028"

url = scrape_yahoo.make_yahoo_json_url(SAMPLE_NFL_GAME)
json_data = scrape_yahoo.get_some_json(url)

In [ ]:
parsed = scrape_yahoo.parse_yahoo_data(json_data)

the parsing from NBA works flawlessly (as far as I can tell)

In [ ]:
parsed

{'game_id': 'nfl.g.20251225028',
 'game_date': '2025-12-25-07:00',
 'away_team': 'Dallas',
 'home_team': 'Washington',
 'pregame_odds': '-9, O/U 50.5',
 'total_over_points': '40.5',
 'total_over_stake_percentage': '73.81',
 'total_over_wager_percentage': '63.44',
 'total_over_odds': -450,
 'total_over_decimal_odds': '1.220',
 'total_over_won': True,
 'total_under_points': '40.5',
 'total_under_stake_percentage': '26.19',
 'total_under_wager_percentage': '36.56',
 'total_under_odds': 310,
 'total_under_decimal_odds': '4.100',
 'total_under_won': False,
 'money_away_odds': -500,
 'money_away_decimal_odds': '1.200',
 'money_away_stake_percentage': '82.91',
 'money_away_wager_percentage': '86.33',
 'money_away_won': True,
 'money_home_odds': 380,
 'money_home_decimal_odds': '4.800',
 'money_home_stake_percentage': '17.09',
 'money_home_wager_percentage': '13.67',
 'money_home_won': False,
 'spread_away_points': '-9',
 'spread_away_odds': -105,
 'spread_away_decimal_odds': '1.950',
 'spread

ok, so I need a new function that will go week by week and get all the game ids

here is week 2 of the current season:

https://sports.yahoo.com/nfl/scoreboard/?confId=&dateRange=2&schedState=2

here is week 2 of the 2022 season:

https://sports.yahoo.com/nfl/scoreboard/?confId=&dateRange=2&schedState=2&scoreboardSeason=2022

`dateRange` is the week of the season. I don't know what `schedState` is for

In [22]:
import cloudscraper
import re
import time
import os

In [6]:
scraper = cloudscraper.create_scraper()
date_url = "https://sports.yahoo.com/nfl/scoreboard/?confId=&dateRange=2&schedState=2&scoreboardSeason=2022"

date_html = scraper.get(date_url).text

In [9]:
foo = set(re.findall(r"nfl\.g\.202[\d]+", date_html))

this gets 25 game ids...

In [11]:
len(foo)

25

In [12]:
foo

{'nfl.g.20220911003',
 'nfl.g.20220911004',
 'nfl.g.20220911006',
 'nfl.g.20220911016',
 'nfl.g.20220911022',
 'nfl.g.20220912026',
 'nfl.g.20220915012',
 'nfl.g.20220918005',
 'nfl.g.20220918006',
 'nfl.g.20220918007',
 'nfl.g.20220918008',
 'nfl.g.20220918009',
 'nfl.g.20220918013',
 'nfl.g.20220918014',
 'nfl.g.20220918018',
 'nfl.g.20220918019',
 'nfl.g.20220918023',
 'nfl.g.20220918025',
 'nfl.g.20220918030',
 'nfl.g.20220918033',
 'nfl.g.20220919002',
 'nfl.g.20220919021',
 'nfl.g.20220925017',
 'nfl.g.20220925027',
 'nfl.g.20260103025'}

the scrape_yahoo logic will ignore a game if we've already fetched it. just gotta make sure that it's only doing the current year (eg 2022 games only)

I should probably refactor the code to be a class, etc. but this is easier.

In [13]:
from scrape_yahoo import *

In [45]:
a = set([4,5])
b = set([4,7])

a.union(b)

{4, 5, 7}

In [46]:
def make_date_url(week, year):
    return f"https://sports.yahoo.com/nfl/scoreboard/?confId=&dateRange={week}&schedState=2&scoreboardSeason={year}"

def get_yahoo_ids_for_date(args):
    """
    NFL VERSION

    fetches date_url and extracts all game ids out of the HTML. takes week and year as args
    """
    (week, year) = args
    scraper = cloudscraper.create_scraper()
    date_url = make_date_url(week, year)
    date_html = scraper.get(date_url).text
    game_ids1 = set(re.findall(f"nfl\.g\.{year}[\d]+", date_html))
    ## some games take place in the next calendar year
    game_ids2 = set(re.findall(f"nfl\.g\.{year + 1}[\d]+", date_html))

    return game_ids1.union(game_ids2)


def fetch_yahoo_data(dir="nfl_scrapes/2022", year=2022):
    """
    NFL version

    fetches all data from `start` to `end` and saves them as JSON in the `dir` directory.
    """
    for week in range(1, 19):
        yahoo_ids = get_yahoo_ids_for_date((week, year))
        time.sleep(2)

        for yahoo_game_id in yahoo_ids:
            cache_path = f"{dir}/{yahoo_game_id}.json"
            if not os.path.exists(cache_path):
                game_url = make_yahoo_json_url(yahoo_game_id)
                #print(f"fetching url {game_url}")
                try:
                    game_json = get_some_json(game_url)

                    with open(cache_path, "w") as f:
                        json.dump(game_json, f)
                except:
                    # this condition happened 3 times in the course of scraping all 
                    # 4 seasons. I didn't investigate why, and rerunning those days
                    # was successful.
                    print(f"failed on {game_url}")
                    # I'm not sure if it's hitting rate limits or what, but might as well
                    # take a little break.
                    time.sleep(10)
                time.sleep(1) # be polite

        print(f"DONE WITH {week}")


I had to re-run this function because there were some errors the first time through.

In [99]:
fetch_yahoo_data(dir="nfl_scrapes/2024", year=2024)

DONE WITH 1
DONE WITH 2
DONE WITH 3
DONE WITH 4
DONE WITH 5
DONE WITH 6
DONE WITH 7
DONE WITH 8
DONE WITH 9
DONE WITH 10
DONE WITH 11
DONE WITH 12
DONE WITH 13
DONE WITH 14
DONE WITH 15
DONE WITH 16
DONE WITH 17
DONE WITH 18


this is crsuty as hell, need to knock out the check on TEAMS

In [ ]:
### FIXME: move this into the class, when I do the refactor
### This makes things VASTLY faster
def preparse_rules():
    parsed = {}
    for k, v in scrape_rules.RULES.items():
        jsonpath_expression = parse(v)
        parsed[k] = jsonpath_expression
    return parsed

def parse_yahoo_data(json_data, filename='', parsed_rules=None):
    """
    takes json data from a single game and parses it

    it returns None for postponed/non-completed games, and games with unrecognized teams (all star games)
    """
    row = {}
    if not parsed_rules:
        parsed_rules = preparse_rules()

    for k, jsonpath_expression in parsed_rules.items():
        ## FIXME: there is probably a way to cache these parsed expressions & save some time
        #jsonpath_expression = parse(v)
        try:
            results = [item.value for item in jsonpath_expression.find(json_data)][0]
            row[k] = results
        except:
            print(f"file: {filename} failed on {jsonpath_expression}")

    if ('total_over_won' in row) and (row['total_over_won'] or row['total_under_won']):
        return row
    else:
        return None # nobody won (postponed game) or yahoo missing data
    
def make_dataframe(json_filenames):
    dataframes = []
    #df = pd.DataFrame(columns=DATAFRAME_CONF.keys())
    parsed_data = None

    parsed_rules = preparse_rules()

    for filename in json_filenames:
        with open(filename, 'r') as f:
            json_data = json.load(f)
        parsed_data = parse_yahoo_data(json_data, filename, parsed_rules)
        if parsed_data: # skip if bad/no data from this file
            dataframes.append(pd.DataFrame({k:[v] for k,v in parsed_data.items()}))

    return pd.concat(dataframes)

In [100]:
filenames = get_cached_filenames("nfl_scrapes/2024")

the jsonpath stuff is so dang slow

there is an error parsing this one game:
`file: nfl_scrapes/2022\nfl.g.20220918018.json failed on $.data.games.[0].gameLineSixPack.[?[Expression(Fields('type') = 'OVER_UNDER'), Expression(Fields('period') = 'FULL_GAME'), Expression(Fields('eventState') = 'PREGAME')]].options.[0].optionDetails.[0].value`

it doesn't have the `FULL_GAME` odds, only the live odds

In [101]:
df = make_dataframe(filenames)

In [102]:
df

,game_id,game_date,away_team,home_team,pregame_odds,total_over_points,total_over_stake_percentage,total_over_wager_percentage,total_over_odds,total_over_decimal_odds,...,spread_away_decimal_odds,spread_away_stake_percentage,spread_away_wager_percentage,spread_away_won,spread_home_points,spread_home_odds,spread_home_decimal_odds,spread_home_stake_percentage,spread_home_wager_percentage,spread_home_won
0,nfl.g.20240905012,2024-09-05-07:00,Baltimore,Kansas City,"-3, O/U 46",46,36.15,77.46,-110,1.910,...,1.870,54.05,59.11,False,-3,-105,1.950,45.95,40.89,True
0,nfl.g.20240906021,2024-09-06-07:00,Green Bay,Philadelphia,"-1.5, O/U 49.5",49.5,82.82,77.01,-110,1.910,...,1.910,61.93,64.7,False,-1.5,-110,1.910,38.07,35.3,True
0,nfl.g.20240908001,2024-09-08-07:00,Pittsburgh,Atlanta,"-4, O/U 42.5",42.5,40.5,85.72,-110,1.910,...,1.910,73.34,83.84,True,-4,-110,1.910,26.66,16.16,False
0,nfl.g.20240908002,2024-09-08-07:00,Arizona,Buffalo,"-6.5, O/U 46",46,51.39,73.54,-110,1.910,...,1.950,59.99,66.9,True,-6.5,-115,1.870,40.01,33.1,False
0,nfl.g.20240908003,2024-09-08-07:00,Tennessee,Chicago,"-4, O/U 43",43,71.91,84.28,-110,1.910,...,1.910,66.22,68.01,False,-4,-110,1.910,33.78,31.99,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,nfl.g.20250105017,2025-01-05-08:00,Buffalo,New England,"-3.5, O/U 36.5",36.5,78.71,85.05,-115,1.870,...,2.000,61.56,69.99,False,3.5,-120,1.830,38.44,30.01,True
0,nfl.g.20250105020,2025-01-05-08:00,Miami,NY Jets,"-2, O/U 39.5",39.5,44.15,51.09,-110,1.910,...,1.910,55.16,70.0,False,-2,-110,1.910,44.84,30.0,True
0,nfl.g.20250105021,2025-01-05-08:00,NY Giants,Philadelphia,"-3, O/U 36",36,69.53,85.0,-110,1.910,...,1.910,52.37,28.23,False,-3,-110,1.910,47.63,71.77,True
0,nfl.g.20250105022,2025-01-05-08:00,San Francisco,Arizona,"-4.5, O/U 43",43,86.21,87.18,-110,1.910,...,1.910,45.73,58.96,False,-4.5,-110,1.910,54.27,41.04,True


FIXME: oh crap, there are only 269 rows and there should be 272. one game had the error listed above, but I don't know about the other two.


In [ ]:
(32/2) * 17

272

ok, now there's a problem that the numeric columns aren't numeric type... not sure why this wasn't a problem with NBA but here we are

In [54]:
df.dtypes


game_id                                 object
game_date                       datetime64[ns]
away_team                               object
home_team                               object
pregame_odds                            object
total_over_points                       object
total_over_stake_percentage             object
total_over_wager_percentage             object
total_over_odds                          int64
total_over_decimal_odds                 object
total_over_won                            bool
total_under_points                      object
total_under_stake_percentage            object
total_under_wager_percentage            object
total_under_odds                         int64
total_under_decimal_odds                object
total_under_won                           bool
money_away_odds                          int64
money_away_decimal_odds                 object
money_away_stake_percentage             object
money_away_wager_percentage             object
money_away_wo

In [103]:
to_numeric = ['spread_away_points', 'spread_away_wager_percentage']
df[to_numeric] = df[to_numeric].apply(pd.to_numeric)

In [104]:
df_fancy = add_spread_columns(df)

In [105]:
df_fancy.spread_popular_won.value_counts()

spread_popular_won
True     140
False    128
Name: count, dtype: int64

in 2025 (so far), the unpopular side has won 53% of the time

In [75]:
134/ (134 + 119)

0.5296442687747036

in 2024, it was only 47.8% winning percentage

In [107]:
128 / (140 + 128)

0.47761194029850745

in 2022, the unpopular side won 56.5% of the time

In [62]:
152 / (152 + 117)

0.5650557620817844

in 2021, it won an incredible 62% of the time

In [83]:
175 / (175 + 107)

0.6205673758865248

In [88]:
df_fancy.spread_popular_won.value_counts()

spread_popular_won
False    175
True     107
Name: count, dtype: int64

in 2023, it won 62.4% of the time. wtf?

In [96]:
169 / (169 + 102)

0.6236162361623616

In [98]:
df_fancy.iloc[1]

game_id                           nfl.g.20230910001
game_date                       2023-09-10 07:00:00
away_team                                  Carolina
home_team                                   Atlanta
pregame_odds                         -3.5, O/U 40.5
total_over_points                              40.5
total_over_stake_percentage                   89.96
total_over_wager_percentage                   88.85
total_over_odds                                -110
total_over_decimal_odds                       1.910
total_over_won                                False
total_under_points                             40.5
total_under_stake_percentage                  10.04
total_under_wager_percentage                  11.15
total_under_odds                               -110
total_under_decimal_odds                      1.910
total_under_won                                True
money_away_odds                                 155
money_away_decimal_odds                       2.550
money_away_s

## gotta catch em all

In [113]:
all_filenames = get_cached_filenames("nfl_scrapes/*")

In [114]:
len(all_filenames)

1373

get all of them in one big df

In [119]:
dataframes = []

for year in range(2021,2026):
    filenames = get_cached_filenames(f"nfl_scrapes/{year}")
    df = make_dataframe(filenames)
    df['season'] = year
    to_numeric = ['spread_away_points', 'spread_away_wager_percentage']
    df[to_numeric] = df[to_numeric].apply(pd.to_numeric)
    dataframes.append(df)

df_all = pd.concat(dataframes)


file: nfl_scrapes/2022\nfl.g.20220918018.json failed on $.data.games.[0].gameLineSixPack.[?[Expression(Fields('type') = 'OVER_UNDER'), Expression(Fields('period') = 'FULL_GAME'), Expression(Fields('eventState') = 'PREGAME')]].options.[0].optionDetails.[0].value
file: nfl_scrapes/2022\nfl.g.20220918018.json failed on $.data.games.[0].gameLineSixPack.[?[Expression(Fields('type') = 'OVER_UNDER'), Expression(Fields('period') = 'FULL_GAME'), Expression(Fields('eventState') = 'PREGAME')]].options.[0].stakePercentage
file: nfl_scrapes/2022\nfl.g.20220918018.json failed on $.data.games.[0].gameLineSixPack.[?[Expression(Fields('type') = 'OVER_UNDER'), Expression(Fields('period') = 'FULL_GAME'), Expression(Fields('eventState') = 'PREGAME')]].options.[0].wagerPercentage
file: nfl_scrapes/2022\nfl.g.20220918018.json failed on $.data.games.[0].gameLineSixPack.[?[Expression(Fields('type') = 'OVER_UNDER'), Expression(Fields('period') = 'FULL_GAME'), Expression(Fields('eventState') = 'PREGAME')]].opti

In [120]:
df_all = add_spread_columns(df_all)

In [126]:
df_all.spread_away_stake_percentage

0    24.31
0    45.16
0    30.38
0    79.35
0    87.48
     ...  
0     51.8
0    72.01
0    48.51
0    47.01
0    58.22
Name: spread_away_stake_percentage, Length: 1343, dtype: object

In [122]:
df_all.spread_popular_won.value_counts()

spread_popular_won
False    758
True     585
Name: count, dtype: int64

In [123]:
758 / (758 + 585)

0.5644080416976918

In [125]:
df_all.groupby("season")["spread_popular_won"].value_counts()

season  spread_popular_won
2021    False                 175
        True                  107
2022    False                 152
        True                  117
2023    False                 169
        True                  102
2024    True                  140
        False                 128
2025    False                 134
        True                  119
Name: count, dtype: int64